In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [13]:
base_model = keras.applications.xception.Xception(weights='imagenet', include_top=True)

# Replace the file path with the testing dataset
raw_dataset = tf.data.TFRecordDataset(['birds-20-eachOf-358.tfrecords'])

feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'birdType': tf.io.FixedLenFeature([], tf.int64)
}

def parse_examples(serialized_examples):
    examples = tf.io.parse_example(serialized_examples, feature_description)
    targets = examples['birdType']
    images = tf.image.resize_with_pad(tf.cast(tf.io.decode_jpeg(examples['image'], channels=3), tf.float32), 299, 299)
    return images, targets

dataset = ('/content/drive/MyDrive/birds-20-eachOf-358.tfrecords')
dataset = raw_dataset.map(parse_examples, num_parallel_calls=10).batch(128)

In [14]:
# Calculate the dataset size manually
dataset_size = tf.data.experimental.cardinality(dataset).numpy()

# Data for the validation set
raw_dataset1 = tf.data.TFRecordDataset(['birds-10-eachOf-358.tfrecords'])

dataset1 = ('/content/drive/MyDrive/birds-10-eachOf-358.tfrecords')
dataset1 = raw_dataset1.map(parse_examples, num_parallel_calls=10).batch(128)

In [15]:
# Calculate the dataset size manually for the validation set
dataset_size1 = tf.data.experimental.cardinality(dataset1).numpy()

# Split the dataset into train and test sets
train_size = int(0.7 * dataset_size)
valid_size = int(dataset_size1)
test_size = dataset_size - train_size

trainset = dataset.take(train_size)
validset = dataset1.take(valid_size)
testset = dataset.skip(train_size)

# Print the number of samples in each set
print("Train set size:", train_size)
print("Validation set size:", valid_size)
print("Test set size:", test_size)


print("Train set size:", train_size)
print("Validation set size:", valid_size)
print("Test set size:", test_size)


def preprocessWithAspectRatio(image, label):
    resized_image = tf.image.resize_with_pad(image, 299, 299)
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

trainPipe = trainset.map(preprocessWithAspectRatio, num_parallel_calls=20).batch(10)
validPipe = validset.map(preprocessWithAspectRatio, num_parallel_calls=20).batch(10)
testPipe = testset.map(preprocessWithAspectRatio, num_parallel_calls=20).batch(10)

base_model = keras.applications.xception.Xception(weights='imagenet', include_top=False)

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(358, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)


# Freezing the bottom layers of the model
for layer in base_model.layers:
    layer.trainable = False

# Setting training parameters
checkpoint_cb = keras.callbacks.ModelCheckpoint('model1.h5', save_best_only=True)
earlyStop_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
learning_rate = 5e-1
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer, metrics=["accuracy"]) 
model.fit(trainPipe,validation_data=validPipe,epochs=5, callbacks=[checkpoint_cb,earlyStop_cb])


model1.save('preprocessing.h5')
# Load the model you saved
model = tf.keras.models.load_model('preprocessing.h5')

# Ensure the metrics included are the ones we wish to evaluate
top5err = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top5')
top10err = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10, name='top10')
top20err = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=20, name='top20')
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', top5err, top10err, top20err])

# Returns the loss and metrics evaluated on the dataset
resp = model.evaluate(dataset)






Train set size: -1
Validation set size: -2
Test set size: -1
Train set size: -1
Validation set size: -2
Test set size: -1
Epoch 1/5


ValueError: ignored